# Trabalho final para a matéria: "Anotação de Textos e Processamento de Linguagem Natural (CIC0087)" 😊

## Reconhecimento de Entidades Nomeadas

### Aluno: Guilherme Braga (17/0162290)

#### [Repositório no Github](https://github.com/gui1080/IA_CIC0087_Reconhecimento_de_Entidades_Nomeadas)

### Pegando os dados originais 🧐

In [164]:
from google.colab import files

# importar o arquivo de disco local, com o nome original "contratos.csv"
uploaded = files.upload()

Saving contratos.csv to contratos (5).csv


### Importando, separando e limpando os dados 🚮


In [165]:
import pandas as pd
import numpy as np
import io
from sklearn.utils import shuffle

# Adicionando em dataframe
df = pd.read_csv(io.BytesIO(uploaded['contratos.csv']))
df = df.replace([np.inf, -np.inf], np.nan)
df = df.reset_index()

df

,index,id_dodf,tipo_rel,id_rel,anotador_rel,timestamp_rel,tipo_ent,id_ent,anotador_ent,timestamp_ent,offset,length,texto
0,0,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,numero_termo_aditivo,275,matheus_santos,2022-03-04T15:34:41Z,372969,20,QUARTO TERMO ADITIVO
1,1,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,numero_contrato,276,matheus_santos,2022-03-04T15:34:41Z,373005,8,002/2018
2,2,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,orgao_contratante,277,matheus_santos,2022-03-04T15:34:41Z,373098,8,DF/RA II
3,3,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,objeto_aditamento_contratual,278,matheus_santos,2022-03-04T15:34:41Z,373238,481,O presente Termo Aditivo objetiva a alteracao ...
4,4,22_20.12.2019.1,REL_ADITAMENTO_CONTRATO,R5,matheus_santos,2022-04-02T22:45:52Z,processo_gdf,434,matheus_santos,2022-04-02T22:45:46Z,373072,16,0131-000016/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6691,6691,8_13.8.2019,REL_EXTRATO_CONTRATO,R5,alexandre_santana,2022-03-08T12:03:42Z,vigencia_contrato,6,alexandre_santana,2022-02-22T02:34:10Z,478144,17,04 (quatro) meses
6692,6692,8_13.8.2019,REL_EXTRATO_CONTRATO,R5,alexandre_santana,2022-03-08T12:03:42Z,valor_contrato,5,alexandre_santana,2022-02-22T02:34:10Z,478173,9,"76.500,00"
6693,6693,8_13.8.2019,REL_EXTRATO_CONTRATO,R5,alexandre_santana,2022-03-08T12:03:42Z,data_assinatura_contrato,149,alexandre_santana,2022-03-08T12:03:37Z,478086,10,07/08/2019
6694,6694,8_13.8.2019,REL_EXTRATO_CONTRATO,R5,alexandre_santana,2022-03-08T12:03:42Z,objeto_contrato,150,alexandre_santana,2022-03-08T12:03:37Z,478106,26,aquisicao de fita eletrica


### Nem sempre é necessária a limpeza total dos dados!

#### ["Aqui entidades numéricas, capitalização e pontuação podem ser importantes. Por isso, pensando em uma tarefa de classificação, faz sentido não adotar alguns dos passos de pré-processamento que normalmente são feitos."](https://colab.research.google.com/drive/1zjcTnNNLlbM3BH3zEAAnQSJjlUmTP8K2?usp=sharing#scrollTo=XP3U0XWKsmSf://)

In [166]:
# 80% vai ser treino, 20% vai ser teste
# 0.8 * 6695 = 5356
# até o índice 5356 vai ser treino, depois disso é teste

# é o dado
x = df.filter(['texto'])

x_train_raw = x.iloc[:5356] 
x_test_raw = x.iloc[5356:]  

# é o resultado do dado
y = df.filter(['id_ent'])

y_train_raw = y.iloc[:5356]
y_test_raw = y.iloc[5356:]


# preciso transformar o texto num numero

frases = df.filter(['texto'])


valores_distintos = y_train_raw['id_ent'].nunique() + 1
valores_distintos_nao_nulos = y_train_raw['id_ent'].count()

print(valores_distintos)              # entidades possíveis
print(valores_distintos_nao_nulos)    # repetições


624
5356


### Conversão para "one-hot codes"

In [167]:
from tensorflow import keras

# conversão para "one-hot codes"

# to_categorical -> vai de 0 até "valores_distintos - 1" como uma possibilidade
# esses dados começam em 1, vou subtrair 1 de tudo, para 0 virar uma categoria

y_train_raw = y_train_raw - 1
y_test_raw = y_test_raw - 1

y_train = keras.utils.to_categorical(y_train_raw, valores_distintos+1)
y_test = keras.utils.to_categorical(y_test_raw, valores_distintos+1)

print(y_train[0])


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

### Testarei inicialmente a stemnminzação, retirada de acentos (para garantir) e a retirada de stopwords da parte do texto. Na retirada de stopwords, não contarei com a retirada de "\n".

In [168]:
!pip install unidecode
import unidecode

In [169]:
# retirada de acentos

frases['texto'] = frases['texto'].apply(lambda x: ' '.join([unidecode.unidecode(termo) for termo in x.split()]))
x_train_raw['texto'] = x_train_raw['texto'].apply(lambda x: ' '.join([unidecode.unidecode(termo) for termo in x.split()]))
x_test_raw['texto'] = x_test_raw['texto'].apply(lambda x: ' '.join([unidecode.unidecode(termo) for termo in x.split()]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [170]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [171]:
# retirada de stopwords

frases['texto'] = frases['texto'].apply(lambda x: ' '.join([termo for termo in x.split() if termo not in stopwords]))
x_train_raw['texto'] = x_train_raw['texto'].apply(lambda x: ' '.join([termo for termo in x.split() if termo not in stopwords]))
x_test_raw['texto'] = x_test_raw['texto'].apply(lambda x: ' '.join([termo for termo in x.split() if termo not in stopwords]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [172]:
nltk.download('rslp')
stemmer = nltk.stem.RSLPStemmer()

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [173]:
# steminização

frases['texto'] = frases['texto'].apply(lambda x: ' '.join(stemmer.stem(termo) for termo in x.split()))
x_train_raw['texto'] = x_train_raw['texto'].apply(lambda x: ' '.join(stemmer.stem(termo) for termo in x.split()))
x_test_raw['texto'] = x_test_raw['texto'].apply(lambda x: ' '.join(stemmer.stem(termo) for termo in x.split()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [174]:
words_raw = set()

for i in range(5356):
    # printing a third element of column
    nossa_string = frases['texto'][i]
    for termo in nossa_string.split():
      words_raw.add(termo)

words_raw = list(words_raw)
num_words = len(words_raw)

def remove_dups(lista):
    """Remove os itens duplicados mantendo a ordem original."""
    return list(dict.fromkeys(lista))

words = remove_dups(words_raw)
num_words = len(words)

# lista de palavras possíveis
num_words

11286

In [175]:
# Limpeza pronta!

# eu tenho uma lista de palavras, "words"
# agora quero mapear cada palavra em x para um valor inteiro nessa lista

def retorna_valor(frase):

  j = 0

  acumulado = 0

  for termo in frase.split():
    j = 0
    for j in range(num_words):
      if(words[j] == termo):
        acumulado = acumulado + j

  return acumulado

x_train = x_train_raw
x_test = x_test_raw

x_train['texto'] = x_train_raw['texto'].apply(retorna_valor)
x_test['texto'] = x_test_raw['texto'].apply(retorna_valor)

# vamos realizar a visualização do que vai servir de treino
x_train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,texto
0,10872
1,2875
2,14394
3,269113
4,440
...,...
5351,6648
5352,54635
5353,18408
5354,3515


In [176]:
# deve-se mapear o menor e o maior valor para um range entre (x e y)

from scipy.interpolate import interp1d

max_index_train = x_train['texto'].max()
max_index_test = x_test['texto'].max()

if(max_index_train > max_index_test):
  max = max_index_train
else:
  max = max_index_test

min_index_train = x_train['texto'].max()
min_index_test = x_test['texto'].max()

if(min_index_train < min_index_test):
  min = min_index_train
else:
  min = min_index_test

print("Maior valor retornado -> " + str(max))
print("Menor valor retornado -> " + str(min))

def new_map(s):
  a = [min, max]
  b = [0, 4096]
  (a1, a2), (b1, b2) = a, b
  return  b1 + ((s - a1) * (b2 - b1) / (a2 - a1))


x_train['texto'] = x_train['texto'].apply(new_map)
x_test['texto'] = x_test['texto'].apply(new_map)


Maior valor retornado -> 2932574
Menor valor retornado -> 2489569


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [177]:
# vamos realizar a visualização do que vai servir de teste

#x_test = x_test.drop(columns=['level_0', 'index'])
x_test

,texto
5356,-18049.388914
5357,-22843.300604
5358,-22967.473649
5359,-23009.977260
5360,-22950.368650
...,...
6691,-22893.912908
6692,-23018.418808
6693,-23005.224844
6694,-22950.692259


In [178]:
x_train

,texto
0,-22917.896891
1,-22991.836715
2,-22885.332671
3,-20530.214729
4,-23014.350592
...,...
5351,-22956.951764
5352,-22513.266586
5353,-22848.219447
5354,-22985.919310


In [179]:
num_words

11286

In [180]:
print(x_train.shape)

(5356, 1)


In [181]:
import numpy 

print(y_train.shape)
y_train = numpy.delete(y_train, (0), axis=1)
y_test = numpy.delete(y_test, (0), axis=1)
print(y_train.shape)

(5356, 625)
(5356, 624)


# RNN

### Nesse tipo de implementação, sequência e ordem importam!

#### Qual deveria ser a proporção escolhida perfeita para essa rede neural é uma pergunta complexa.


In [182]:
x_train = np.expand_dims(x_train, axis=-1)
y_train = np.expand_dims(y_train, axis=1)
print(x_train.shape)
print(y_train.shape)

(5356, 1, 1)
(5356, 1, 624)


In [183]:
print(x_test.shape)
print(y_test.shape)


print(x_test.shape)
print(y_test.shape)

(1340, 1)
(1340, 624)
(1340, 1)
(1340, 624)


In [184]:
#nx_train = x_train.to_numpy()
#ny_train = y_train.to_numpy()
#nx_test = x_test.to_numpy()
#ny_test = y_test.to_numpy()

#nx_train.reshape(1, 5356, 1)
#ny_train.reshape(1, 5356, 625)
#nx_test.reshape(1, 5356, 1)
#zny_test.reshape(1, 5356, 625)

# 28 sequencias de 28 elementos

#x_train = np.expand_dims(x_train, axis=-1)
#y_train = np.expand_dims(y_train, axis=-1)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

# Input(shape = (T,D)) for RNN
# T = T is length of time sequence
# D = D is the number of features

# x_train.shape[1:] => (1,)

# input_shape=(x_train.shape[1:])
# input_shape=(5356, 1)

model = Sequential()

model.add(LSTM(2496, input_shape=(1, 1), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(2496, activation='relu', return_sequences=True))
model.add(Dropout(0.1))

model.add(Dense(1248, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(valores_distintos, activation='softmax'))




### Treinamento da RNN



In [185]:
print(x_train.shape)
print(y_train.shape)

x_test = np.expand_dims(x_test, axis=-1)
y_test = np.expand_dims(y_test, axis=1)

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
   loss='binary_crossentropy',
   optimizer=opt,
   metrics=['accuracy'],
)

model.fit(x_train,
          y_train,
          epochs=10,
          validation_data=(x_test, y_test))

(5356, 1, 1)
(5356, 1, 624)
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


168/168 [==============================] - 147s 855ms/step - loss: 0.1378 - accuracy: 0.0015 - val_loss: 0.0123 - val_accuracy: 0.0030
Epoch 2/5
168/168 [==============================] - 142s 847ms/step - loss: 0.0163 - accuracy: 0.0028 - val_loss: 0.0119 - val_accuracy: 0.0030
Epoch 3/5
168/168 [==============================] - 144s 858ms/step - loss: 0.0147 - accuracy: 0.0021 - val_loss: 0.0121 - val_accuracy: 0.0037
Epoch 4/5
168/168 [==============================] - 142s 846ms/step - loss: 0.0142 - accuracy: 0.0024 - val_loss: 0.0136 - val_accuracy: 0.0045
Epoch 5/5
168/168 [==============================] - 144s 857ms/step - loss: 0.0140 - accuracy: 0.0015 - val_loss: 0.0122 - val_accuracy: 0.0015


In [186]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.012158951722085476
Test accuracy: 0.0014925373252481222
